Note:
#Ignore Gender since it has lot of bias

Nan fields:
state,self_employed,work_interfere,comments

Note:

#Gender

##For Training data:

Enby,fluid,Genderqueer,Trans-female,Nah,Androgyne,Agender,male leaning androgynous,Trans woman,Neuter,Female (trans),queer,non-binary-  3
Female,female,Cis Female,F,Woman,f,Femake,Female ,woman,cis-female/femme,Female (cis) -2
M,Male,male,m,Male-ish,maile,something kinda male?,Cis Male,Mal,Male ,Make,Guy (-ish) ^_^,Man,msle,Mail,cis male - 1

##For Test data:

Male,male,m,Malr,Male ,Cis Man - 1
female,F,Female,Woman,femail-2
p,Female (trans),ostensibly male,unsure what that really means - 3


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
#read data from csv
df = pd.read_csv('trainms.csv')
df.columns

In [ ]:
#Gender Validation
temp = df.Gender
temp = temp.replace(to_replace=["A little about you","something kinda male?","Enby","fluid","Genderqueer","Trans-female","Nah","Androgyne","Agender","male leaning androgynous","Trans woman","Neuter","Female (trans)","queer","non-binary"],value="T")
temp = temp.replace(["Female","female","Cis Female","F","Woman","f","Femake","Female ","woman","cis-female/femme","Female (cis)"],'F')
temp = temp.replace(["M","Male","male","m","Male-ish","maile","Cis Male","Mal","Male ","Make","Guy (-ish) ^_^","Man","msle","Mail","cis male"],'M')
temp.unique()

In [159]:
#Training the classifier

features = df.drop(columns=['treatment','state','self_employed','work_interfere','comments','s.no','Timestamp'])
y = df.treatment

le = preprocessing.LabelEncoder()
features = features.apply(le.fit_transform)
labels,uniques = pd.factorize(y)

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=-1, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(features, labels)

In [162]:
#Verify predictions
train_predicted = clf.predict(features)

idx = pd.Index(labels,name="sss")
print(idx.value_counts())
accuracy = accuracy_score(labels,train_predicted)
print(accuracy)

print(confusion_matrix(labels,train_predicted))

1    508
0    492
Name: sss, dtype: int64
0.988
[[487   5]
 [  7 501]]


In [ ]:
#Trying to read and view test data
test = pd.read_csv('testms.csv')

In [ ]:
#Validate test gender
temp = test.Gender
temp = temp.replace(to_replace=["Male","male","m","Malr","Male ","Cis Man"],value='M')
temp = temp.replace(to_replace=["female","F","Female","Woman","femail","f"],value='F')
temp = temp.replace(to_replace=["p","Female (trans)","ostensibly male, unsure what that really means"],value='T')
test.Gender = temp

In [151]:
#predict testing data
test_features = test.drop(columns=['state','self_employed','work_interfere','comments','s.no','Timestamp'])
le = preprocessing.LabelEncoder()
test_features = test_features.apply(le.fit_transform)

pred = clf.predict(test_features)

In [ ]:
# Submission generation
index = [str(i) for i in range(1,len(pred)+1)]
values = ['Yes' if val==0 else 'No' for i,val in enumerate(pred)]
submission_result = pd.DataFrame(values, index=index,columns=['treatment'])
submission_result.index.name = "s.no"

file_name="submission.csv"
submission_result.to_csv(file_name, sep=',')